In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
#ccba
ccba = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_cdtx0001_full_hashed.csv')
#custinfo
cdtx2 = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_custinfo_full_hashed.csv')
#cdtx
dp = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_remit1_full_hashed.csv')
#alert_time
alert_time = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_x_alert_date.csv')
#y
y = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\train_y_answer.csv')
#案件名單
doc = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\預測的案件名單及提交檔案範例.csv')

In [3]:
cdtx['country'] = (cdtx['country'] == 130).astype(int)
cdtx['cur_type'] = (cdtx['cur_type'] == 47).astype(int)

In [4]:
cdtx['country'] = cdtx['country'].apply(lambda x: 'TW' if x == 1 else 'FN')
cdtx['cur_type'] = cdtx['cur_type'].apply(lambda x: 'TWD' if x == 1 else 'FND')

In [5]:
doc_merged = cdtx2.merge(doc, on='alert_key', how='right').merge(alert_time, on='alert_key', how='left')
doc_merged = doc_merged[~doc_merged['cust_id'].isnull()]
max_doc = doc_merged.groupby('cust_id')['date'].max().reset_index()
max_doc.rename(columns={'date':'max_date'},inplace=True)
doc_merged = doc_merged.merge(max_doc, on='cust_id', how='left')
prev_list = doc_merged[doc_merged['max_date']>doc_merged['date']]['alert_key'].tolist()

In [6]:
alert_cdtx = cdtx[cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_cdtx = cdtx[~cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_cdtx['y'] = 0

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
sar_idx = train_cdtx[train_cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
train_cdtx.loc[sar_idx, 'y'] = 1

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
#Data Cleansing
#SAR戶僅留前X天
def sar_pred_d(x, day):
  prev_d = x.groupby('cust_id')['date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x

train_cdtx = sar_pred_d(train_cdtx, 30)
alert_cdtx = sar_pred_d(alert_cdtx, 30)


In [9]:
train_cdtx = pd.merge(train_cdtx, cdtx2)
alert_cdtx = pd.merge(alert_cdtx, cdtx2)

In [11]:
ccba_transfer = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\ccba_transfered.csv')

In [19]:
ccba_transfer.columns = ['cust_id', 'lupay', 'date', 'cycam', 'usgam', 'clamt', 'csamt', 'inamt', 'cucsm', 'cucah']

In [22]:
train_cdtx = train_cdtx.merge(ccba_transfer)
alert_cdtx = alert_cdtx.merge(ccba_transfer)

In [23]:
train_cdtx

,cust_id,date,country,cur_type,amt,y,alert_key,risk_rank,occupation_code,total_asset,AGE,lupay,cycam,usgam,clamt,csamt,inamt,cucsm,cucah
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,TW,TWD,237.0,0,195913,1,17.0,32758.0,3,4898.0,134614.0,19552.0,0.0,0.0,3148.0,3813.0,0.0
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,TW,TWD,412.0,0,195913,1,17.0,32758.0,3,4898.0,134614.0,19552.0,0.0,0.0,3148.0,3813.0,0.0
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,269235,3,4.0,629787.0,3,94538.0,304961.0,75450.0,0.0,0.0,26515.0,27557.0,0.0
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,270967,3,4.0,629787.0,3,94538.0,304961.0,75450.0,0.0,0.0,26515.0,27557.0,0.0
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,271556,3,4.0,629787.0,3,94538.0,304961.0,75450.0,0.0,0.0,26515.0,27557.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,TW,TWD,229.0,0,303894,1,14.0,458455.0,5,19766.0,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,TW,TWD,1112.0,0,303894,1,14.0,458455.0,5,19766.0,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,TW,TWD,59.0,0,303894,1,14.0,458455.0,5,19766.0,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,TW,TWD,156.0,0,303894,1,14.0,458455.0,5,19766.0,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0


In [33]:
alert_cdtx

,cust_id,date,country,cur_type,amt,alert_key,risk_rank,occupation_code,total_asset,AGE,...,cust_id_1_1,alert_key_1_1,total_amt_rolling_7,cust_id_1,alert_key_1,total_amt_rolling_14,country_amt_diff_ratio_x,cur_amt_diff_ratio_x,country_amt_diff_ratio_y,cur_amt_diff_ratio_y
0,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,359,TW,TWD,2245.0,356608,1,19.0,493333.0,4,...,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0.894737,0.999951,0.999902,0.999902
1,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,359,TW,TWD,2245.0,358040,1,19.0,493333.0,4,...,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0.894737,0.999951,0.999902,0.999902
2,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,359,TW,TWD,2245.0,363322,1,19.0,493333.0,4,...,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0.894737,0.999951,0.999902,0.999902
3,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,365,TW,TWD,1023.0,356608,1,19.0,493333.0,4,...,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0.894737,0.999951,0.999902,0.999902
4,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,365,TW,TWD,1023.0,358040,1,19.0,493333.0,4,...,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,356608.0,NaN,0.894737,0.999951,0.999902,0.999902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338730,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,380,TW,TWD,3038.0,355182,1,18.0,824.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.857143,0.999979,0.999958,0.999958
338731,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,381,TW,TWD,1499.0,355182,1,18.0,824.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.857143,0.999979,0.999958,0.999958
338732,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,388,TW,TWD,468.0,355182,1,18.0,824.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.857143,0.999979,0.999958,0.999958
338733,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,386,TW,TWD,273.0,360492,1,19.0,2406614.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.998864,0.997727,0.997727


In [24]:
class renamer():
    def __init__(self):
        self.d = dict()
    def __call__(self, x):
        if x not in self.d:
            self.d[x] =  0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

In [25]:
def total_amt_rolling_3(data):
    total_amt_rolling_3 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(3, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_3.columns = ['cust_id', 'alert_key', 'total_amt_rolling_3']
    on_day_total_amt = train_cdtx.groupby(['cust_id', 'date', 'alert_key'])['amt'].sum().reset_index()
    on_day_total_amt.columns = ['cust_id', 'date', 'alert_key', 'on_day_total_amt']
    data = pd.concat([data, total_amt_rolling_3], axis=1)
    data = data.rename(columns=renamer())  
    data = pd.merge(data, on_day_total_amt, how='left') 
    data['total_amt_rolling_3'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_3']) else x['total_amt_rolling_3'], axis=1)
    return data

train_cdtx = total_amt_rolling_3(train_cdtx)
alert_cdtx = total_amt_rolling_3(alert_cdtx)

In [26]:
def total_amt_rolling_7(data):
    total_amt_rolling_7 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(7, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_7.columns = ['cust_id', 'alert_key', 'total_amt_rolling_7']
    data = pd.concat([data, total_amt_rolling_7], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_7'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_7']) else x['total_amt_rolling_7'], axis=1)
    return data
train_cdtx = total_amt_rolling_7(train_cdtx)
alert_cdtx = total_amt_rolling_7(alert_cdtx)

In [27]:
def total_amt_rolling_14(data):
    total_amt_rolling_14 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(14, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_14.columns = ['cust_id', 'alert_key', 'total_amt_rolling_14']
    data = pd.concat([data, total_amt_rolling_14], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_14'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_14']) else x['total_amt_rolling_14'], axis=1)
    return data
train_cdtx = total_amt_rolling_14(train_cdtx)
alert_cdtx = total_amt_rolling_14(alert_cdtx)

In [28]:
def amt_diff_func(data):
  session_amt_diff = data.groupby(['cust_id','country'])['amt'].count().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='country', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['country_amt_diff_ratio'] = \
    abs(session_amt_diff['TW'] - session_amt_diff['FN']) / abs(session_amt_diff['TW'] + session_amt_diff['FN'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','country_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')
  

  session_amt_diff = data.groupby(['cust_id','cur_type'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='cur_type', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['cur_amt_diff_ratio'] = \
    session_amt_diff['TWD'] / abs(session_amt_diff['TWD'] + session_amt_diff['FND'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','cur_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')

  return data

In [29]:
train_cdtx = amt_diff_func(train_cdtx)
alert_cdtx = amt_diff_func(alert_cdtx)

In [30]:
def amt_diff_func(data):
  session_amt_diff = data.groupby(['cust_id','country'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='country', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['country_amt_diff_ratio'] = \
    abs(session_amt_diff['TW'] - session_amt_diff['FN']) / abs(session_amt_diff['TW'] + session_amt_diff['FN'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','country_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')
  

  session_amt_diff = data.groupby(['cust_id','cur_type'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='cur_type', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['cur_amt_diff_ratio'] = \
    abs(session_amt_diff['TWD'] - session_amt_diff['FND']) / abs(session_amt_diff['TWD'] + session_amt_diff['FND'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','cur_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')

  return data

In [31]:
train_cdtx = amt_diff_func(train_cdtx)
alert_cdtx = amt_diff_func(alert_cdtx)

In [90]:
train_cdtx = train_cdtx.rename(columns=renamer())
alert_cdtx = alert_cdtx.rename(columns=renamer())

In [91]:
train_cdtx['country'] = train_cdtx['country'].astype('category')
train_cdtx['cur_type'] = train_cdtx['cur_type'].astype('category')

alert_cdtx['country'] = alert_cdtx['country'].astype('category')
alert_cdtx['cur_type'] = alert_cdtx['cur_type'].astype('category')

train_cdtx['risk_rank'] = train_cdtx['risk_rank'].astype('category')
train_cdtx['occupation_code'] = train_cdtx['occupation_code'].astype('category')
train_cdtx['AGE'] = train_cdtx['AGE'].astype('category')
alert_cdtx['risk_rank'] = alert_cdtx['risk_rank'].astype('category')
alert_cdtx['occupation_code'] = alert_cdtx['occupation_code'].astype('category')
alert_cdtx['AGE'] = alert_cdtx['AGE'].astype('category')

In [92]:
cat_cols = ['cust_id', 'date', 'risk_rank', 'occupation_code', 'AGE']
train_cdtx = train_cdtx.sort_values(['cust_id', 'date'])
train_cdtx['risk_rank'] = train_cdtx['risk_rank'].ffill()
train_cdtx['occupation_code'] = train_cdtx['occupation_code'].ffill()
train_cdtx['AGE'] = train_cdtx['AGE'].ffill()

In [93]:
train_cdtx

,cust_id,date,country,cur_type,amt,y,alert_key,risk_rank,occupation_code,total_asset,...,country_amt_diff_ratio_y,cur_amt_diff_ratio_y,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,ratio_ccba_ratio_amt,ratio_ccba_ratio_am,ratio_ccba_ratio_am_last,ratio_ccba_ratio_am_14
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,TW,TWD,237.0,0,195913,1,17.0,32758.0,...,0.996923,0.996923,0.007235,0.007235,0.007235,0.007235,0.356025,0.145245,0.250511,3.700324e-07
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,TW,TWD,412.0,0,195913,1,17.0,32758.0,...,0.996923,0.996923,0.012577,0.012577,0.012577,0.012577,0.356025,0.145245,0.250511,6.432631e-07
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,269235,3,4.0,629787.0,...,0.999991,0.999991,0.036320,0.036320,0.036320,0.036320,0.716660,0.247409,1.252989,4.813813e-07
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,270967,3,4.0,629787.0,...,0.999991,0.999991,0.036320,0.036320,0.036320,0.036320,0.716660,0.247409,1.252989,4.813813e-07
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,271556,3,4.0,629787.0,...,0.999991,0.999991,0.036320,0.036320,0.036320,0.036512,0.716660,0.247409,1.252989,4.813813e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,TW,TWD,229.0,0,303894,1,14.0,458455.0,...,0.976025,0.999951,0.000500,0.000500,0.000500,0.000500,0.140744,0.033727,0.749848,1.894931e-08
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,TW,TWD,1112.0,0,303894,1,14.0,458455.0,...,0.976025,0.999951,0.002426,0.002426,0.002426,0.002426,0.140744,0.033727,0.749848,9.201586e-08
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,TW,TWD,59.0,0,303894,1,14.0,458455.0,...,0.976025,0.999951,0.000129,0.000129,0.000129,0.000129,0.140744,0.033727,0.749848,4.882136e-09
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,TW,TWD,156.0,0,303894,1,14.0,458455.0,...,0.976025,0.999951,0.000340,0.000340,0.000340,0.000340,0.140744,0.033727,0.749848,1.290870e-08


In [109]:
train_cdtx['ratio_total_aum_7'] = train_cdtx.total_amt_rolling_7 /train_cdtx.total_asset
train_cdtx['ratio_total_aum_1'] = train_cdtx.on_day_total_amt  /train_cdtx.total_asset 
train_cdtx['ratio_total_aum_14'] = train_cdtx.total_amt_rolling_14/train_cdtx.total_asset
train_cdtx['ratio_total_aum_3'] = train_cdtx.total_amt_rolling_3  /train_cdtx.total_asset
alert_cdtx['ratio_total_aum_7'] = alert_cdtx.total_amt_rolling_7 /alert_cdtx.total_asset
alert_cdtx['ratio_total_aum_1'] = alert_cdtx.on_day_total_amt  /alert_cdtx.total_asset 
alert_cdtx['ratio_total_aum_14'] = alert_cdtx.total_amt_rolling_14/alert_cdtx.total_asset
alert_cdtx['ratio_total_aum_3'] = alert_cdtx.total_amt_rolling_3  /alert_cdtx.total_asset

In [110]:
train_cdtx['ratio_ccba_ratio_amt'] = (train_cdtx.inamt + train_cdtx.cucsm)/train_cdtx.usgam
train_cdtx['ratio_ccba_ratio_am'] = train_cdtx.usgam/train_cdtx.cycam
train_cdtx['ratio_ccba_ratio_am_last'] = (train_cdtx.lupay)/train_cdtx.usgam
train_cdtx['ratio_ccba_ratio_am_14'] = (train_cdtx.total_amt_rolling_14)/train_cdtx.usgam

alert_cdtx['ratio_ccba_ratio_amt'] = (alert_cdtx.inamt + alert_cdtx.cucsm)/alert_cdtx.usgam
alert_cdtx['ratio_ccba_ratio_am'] = alert_cdtx.usgam/alert_cdtx.cycam
alert_cdtx['ratio_ccba_ratio_am_last'] = (alert_cdtx.lupay)/alert_cdtx.usgam
alert_cdtx['ratio_ccba_ratio_am_14'] = (alert_cdtx.total_amt_rolling_14)/alert_cdtx.usgam

In [125]:
(train_cdtx.total_amt_rolling_14)/train_cdtx.usgam

0         0.012122
1         0.021072
2         0.303168
3         0.303168
4         0.303168
            ...   
378484    0.008687
378485    0.042185
378486    0.002238
378487    0.005918
378488    0.020332
Length: 378489, dtype: float64

In [111]:
alert_cdtx.replace([np.inf, -np.inf], 1, inplace=True)

In [112]:
train_cdtx.columns

Index(['cust_id', 'date', 'country', 'cur_type', 'amt', 'y', 'alert_key',
       'risk_rank', 'occupation_code', 'total_asset', 'AGE', 'lupay', 'cycam',
       'usgam', 'clamt', 'csamt', 'inamt', 'cucsm', 'cucah', 'cust_id_1',
       'alert_key_1', 'total_amt_rolling_3', 'on_day_total_amt', 'cust_id_1_1',
       'alert_key_1_1', 'total_amt_rolling_7', 'cust_id_1_1', 'alert_key_1_1',
       'total_amt_rolling_14', 'country_amt_diff_ratio_x',
       'cur_amt_diff_ratio_x', 'country_amt_diff_ratio_y',
       'cur_amt_diff_ratio_y', 'ratio_total_aum_7', 'ratio_total_aum_1',
       'ratio_total_aum_14', 'ratio_total_aum_3', 'ratio_ccba_ratio_amt',
       'ratio_ccba_ratio_am', 'ratio_ccba_ratio_am_last',
       'ratio_ccba_ratio_am_14'],
      dtype='object')

In [113]:
train_cdtx.loc[:, ~train_cdtx.columns.isin(cat_cols+['country','cur_type'])].fillna(0,inplace=True)
alert_cdtx.loc[:, ~alert_cdtx.columns.isin(cat_cols+['country','cur_type'])].fillna(0,inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [127]:
train_col = ['cust_id', 'date', 'alert_key', 'amt', 'country', 'cur_type',
       'risk_rank', 'occupation_code', 'total_asset', 'AGE', 'lupay', 'cycam',
       'usgam', 'clamt', 'csamt', 'inamt', 'cucsm', 'cucah',
       'total_amt_rolling_3', 'on_day_total_amt',
       'total_amt_rolling_7', 'total_amt_rolling_14', 'country_amt_diff_ratio_x',
       'cur_amt_diff_ratio_x', 'country_amt_diff_ratio_y',
       'cur_amt_diff_ratio_y', 'ratio_total_aum_7', 'ratio_total_aum_1',
       'ratio_total_aum_14', 'ratio_total_aum_3', 'ratio_ccba_ratio_amt',
       'ratio_ccba_ratio_am', 'ratio_ccba_ratio_am_last',
        'ratio_ccba_ratio_am_14'
       , 'y']
x_col = train_col[3:-1]
train_data = train_cdtx[train_col]
train_data

,cust_id,date,alert_key,amt,country,cur_type,risk_rank,occupation_code,total_asset,AGE,...,cur_amt_diff_ratio_y,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,ratio_ccba_ratio_amt,ratio_ccba_ratio_am,ratio_ccba_ratio_am_last,ratio_ccba_ratio_am_14,y
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,195913,237.0,TW,TWD,1,17.0,32758.0,3,...,0.996923,0.007235,0.007235,0.007235,0.007235,0.356025,0.145245,0.250511,0.012122,0
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,195913,412.0,TW,TWD,1,17.0,32758.0,3,...,0.996923,0.012577,0.012577,0.012577,0.012577,0.356025,0.145245,0.250511,0.021072,0
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,269235,508.0,TW,TWD,3,4.0,629787.0,3,...,0.999991,0.036320,0.036320,0.036320,0.036320,0.716660,0.247409,1.252989,0.303168,0
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,270967,508.0,TW,TWD,3,4.0,629787.0,3,...,0.999991,0.036320,0.036320,0.036320,0.036320,0.716660,0.247409,1.252989,0.303168,0
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,271556,508.0,TW,TWD,3,4.0,629787.0,3,...,0.999991,0.036320,0.036320,0.036320,0.036512,0.716660,0.247409,1.252989,0.303168,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,303894,229.0,TW,TWD,1,14.0,458455.0,5,...,0.999951,0.000500,0.000500,0.000500,0.000500,0.140744,0.033727,0.749848,0.008687,0
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,303894,1112.0,TW,TWD,1,14.0,458455.0,5,...,0.999951,0.002426,0.002426,0.002426,0.002426,0.140744,0.033727,0.749848,0.042185,0
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,303894,59.0,TW,TWD,1,14.0,458455.0,5,...,0.999951,0.000129,0.000129,0.000129,0.000129,0.140744,0.033727,0.749848,0.002238,0
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,303894,156.0,TW,TWD,1,14.0,458455.0,5,...,0.999951,0.000340,0.000340,0.000340,0.000340,0.140744,0.033727,0.749848,0.005918,0


In [128]:
train_data = train_data.drop_duplicates()

In [129]:
Y = train_data['y']
X = train_data[x_col]

In [131]:
X.describe()

,amt,total_asset,lupay,cycam,usgam,clamt,csamt,inamt,cucsm,cucah,...,country_amt_diff_ratio_y,cur_amt_diff_ratio_y,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,ratio_ccba_ratio_amt,ratio_ccba_ratio_am,ratio_ccba_ratio_am_last,ratio_ccba_ratio_am_14
count,3.421000e+05,3.421000e+05,3.421000e+05,3.421000e+05,3.421000e+05,342100.000000,342100.000000,3.421000e+05,3.421000e+05,342100.000000,...,342100.000000,342100.000000,3.420950e+05,3.420910e+05,3.420940e+05,3.420950e+05,3.408310e+05,342100.000000,3.417800e+05,3.420440e+05
mean,5.126566e+03,6.979171e+05,7.530613e+05,2.952354e+05,1.267065e+05,95.605542,1.129377,4.136227e+03,7.714822e+05,129.300038,...,0.886095,0.937944,inf,inf,inf,inf,NaN,0.460945,inf,inf
std,4.459802e+04,1.634205e+06,1.189024e+06,4.630752e+05,2.716172e+05,4617.768636,4076.643342,2.008711e+04,1.225890e+06,5809.875835,...,0.215679,0.175668,NaN,NaN,NaN,NaN,NaN,0.470771,NaN,NaN
min,0.000000e+00,0.000000e+00,0.000000e+00,1.632700e+04,-5.437510e+05,0.000000,-265796.000000,-5.353000e+03,-2.154700e+05,0.000000,...,0.004381,0.012112,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-inf,-9.472073,-2.301716e+05,-2.301706e+05
25%,1.810000e+02,6.384600e+04,3.545300e+04,1.026130e+05,2.887300e+04,0.000000,0.000000,0.000000e+00,3.870100e+04,0.000000,...,0.893100,0.998580,5.118050e-03,3.366044e-03,6.895784e-03,2.579632e-03,8.843974e-01,0.132394,6.955901e-01,2.555243e-02
50%,5.230000e+02,2.015980e+05,1.616460e+05,2.019140e+05,6.062200e+04,0.000000,0.000000,0.000000e+00,1.456090e+05,0.000000,...,0.997291,0.999928,2.760539e-02,5.484448e-02,4.651884e-02,1.232155e-02,1.089942e+00,0.416955,1.847628e+00,1.351876e-01
75%,2.010000e+03,7.977560e+05,1.213901e+06,3.120110e+05,1.753130e+05,0.000000,0.000000,0.000000e+00,1.304244e+06,0.000000,...,0.999903,0.999999,2.330542e-01,4.917614e-01,3.642270e-01,1.238234e-01,6.778450e+00,0.858854,7.504914e+00,6.480000e-01
max,1.087809e+07,7.386321e+07,4.629297e+06,4.513357e+07,4.087168e+07,507272.000000,498099.000000,1.568301e+06,2.675697e+07,507272.000000,...,1.000000,1.000000,inf,inf,inf,inf,inf,18.948373,inf,inf


In [132]:
X.replace([np.inf, -np.inf], 1, inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [134]:
#model training 1
#逐筆交易處理
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0,
        enable_categorical=True,
        use_label_encoder=False,
        predictor= "gpu_predictor")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#previous 30days label of 1
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 73.76%
#0.9921875
#0.5869618097786427

F1 score: 99.96%
0.9999455040871935
0.9992920161202483


In [135]:
pd.DataFrame({'columns':list(X.columns),'score':model.feature_importances_})

,columns,score
0,amt,0.003377
1,country,0.000142
2,cur_type,0.000382
3,risk_rank,0.007115
4,occupation_code,0.041847
5,total_asset,0.002972
6,AGE,0.006277
7,lupay,0.006625
8,cycam,0.006628
9,usgam,0.022165


In [136]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.05:
        return 1
    elif 0.05 < proba <= 0.1:
        return 2
    elif 0.1 < proba <= 0.15:
        return 3
    elif 0.15 < proba <= 0.2:
        return 4
    elif 0.2 < proba <= 0.25:
        return 5
    elif 0.25 < proba <= 0.3:
        return 6
    elif 0.3 < proba <= 0.35:
        return 7
    elif 0.35 < proba <= 0.4:
        return 8
    elif 0.4 < proba <= 0.45:
        return 9
    elif 0.45 < proba <= 0.5:
        return 10
    elif 0.5 < proba <= 0.55:
        return 11
    elif 0.55 < proba <= 0.6:
        return 12
    elif 0.6 < proba <= 0.65:
        return 13
    elif 0.65 < proba <= 0.7:
        return 14
    elif 0.7 < proba <= 0.75:
        return 15
    elif 0.75 < proba <= 0.8:
        return 16
    elif 0.8 < proba <= 0.85:
        return 17
    elif 0.85 < proba <= 0.9:
        return 18
    elif 0.9 < proba <= 0.95:
        return 19
    elif 0.95 < proba <= 1:
        return 20

In [137]:
train_data['proba'] = model.predict_proba(X)[:,1]

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [138]:
result = train_data[['cust_id','proba','y']]
result['level'] = result["proba"].map(mapping)
result = result[['cust_id', 'level', 'y']]
result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
result.fillna(0, inplace=True)
result = result.div(result.sum(axis=1), axis=0)
result = result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(result.columns[1:])):
  result[col] = 0.0
result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
result = result.merge(train_data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
result.pop('cust_id')

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0       0015cc01f553512007705edf855baa16f1b57a16fb107e...
1       002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...
2       002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...
3       0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...
4       0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...
                              ...                        
3516    ffa5bcc6f88df5c33e809da4daa54de0031803f37d8f11...
3517    ffae7fef923c78298cd647a3d573e48cc1404b07ac794a...
3518    ffdbc1a5135cb6feddf7355e5656abf6973f21639c7b9b...
3519    ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...
3520    ffeb846f2294ec4397f800736dc1e14eb754da5425398d...
Name: cust_id, Length: 3521, dtype: object

In [139]:
#model training 2
#歸戶判斷是否報SAR
result_col = list(result.columns)
result_col.remove('y')
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)
model_2nd = XGBClassifier()
model_2nd.fit(X_train, y_train)
y_pred = model_2nd.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 36.36%
#0.7142857142857143
#0.24390243902439024

F1 score: 97.96%
0.96
1.0


In [140]:
#Submission
alert_x = alert_cdtx[x_col]
alert_data = alert_cdtx
alert_data['proba'] = model.predict_proba(alert_x)[:,1]
alert_result = alert_data[['cust_id','proba']]
alert_result['level'] = alert_result["proba"].map(mapping)
alert_result = alert_result[['cust_id', 'level']]
alert_result['cnt'] = 1
alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
alert_result.fillna(0, inplace=True)
alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
alert_result = alert_result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(alert_result.columns[1:])):
  alert_result[col] = 0.0
alert_result = alert_result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
#alert_result.pop('cust_id')
alert_pred = model_2nd.predict_proba(alert_result[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
# evaluate predictions
final = alert_result[['cust_id']]
final['probability'] = alert_pred[:,1]
final

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


level,cust_id,probability
0,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,0.001524
1,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.001524
2,02e2cb9bb3a904b4e44b7e40f54d536ca5941db440ba00...,0.001524
3,035e2ebcd5c23f4dd0f1fc81a1fca8afd3121f7431bb58...,0.001524
4,03d4cbfcb065501bebb2b75ea4df2b779df0192a183af3...,0.001524
...,...,...
419,fec476c1d12e20806dc384748798a07bfd972ed0626890...,0.001524
420,fee6ee67b6db61c5a7788074e412b0c0ac77b3762b3b98...,0.001524
421,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,0.001524
422,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,0.001524


In [141]:
final = alert_time.merge(cdtx2[['alert_key', 'cust_id']].merge(final, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]

In [142]:
final.fillna(0, inplace=True)
final

,alert_key,probability
0,352342,0.001524
1,352866,0.000000
2,352696,0.001524
3,352330,0.000000
4,352683,0.000000
...,...,...
1840,364472,0.001524
1841,364788,0.000000
1842,364673,0.000000
1843,364626,0.000000


In [143]:
doc = doc[['alert_key']]
doc

,alert_key
0,357307
1,376329
2,373644
3,357668
4,354443
...,...
3845,364485
3846,363155
3847,368710
3848,358067


In [144]:
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final

,alert_key,probability
0,357307,0.001524
1,376329,0.000000
2,373644,0.000000
3,357668,0.001524
4,354443,0.001524
...,...,...
3845,364485,0.001524
3846,363155,0.001524
3847,368710,0.000000
3848,358067,0.000000


In [145]:
final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0

In [147]:
final.to_csv('ccba+cdtx+custinfo.csv')

In [148]:
final[final['probability']>0]

,alert_key,probability
0,357307,0.001524
3,357668,0.001524
5,361128,0.744918
10,362463,0.001524
29,357471,0.001524
...,...,...
3803,361028,0.296562
3806,354423,0.001524
3841,353051,0.001524
3844,360534,0.001524
